In [ ]:
from autumn.core.inputs.database import get_input_db
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib import pyplot
import pandas as pd
import warnings

from summer.utils import ref_times_to_dti

from autumn.core.plots.utils import REF_DATE
from autumn.core.project import get_project
from autumn.core.utils.display import pretty_print


In [ ]:
pyplot.style.use("ggplot")
warnings.filterwarnings("ignore")

region = "malaysia"
project = get_project("sm_sir", region)

In [ ]:
baseline_params = project.param_set.baseline
custom_params = project.param_set.baseline

In [ ]:
model = project.run_baseline_model(custom_params)
derived_df = model.get_derived_outputs_df()

model_start_time = ref_times_to_dti(REF_DATE, [custom_params["time"]["start"]])[0]
model_end_time = ref_times_to_dti(REF_DATE, [custom_params["time"]["end"]])[0]

In [ ]:
targets_dict = {
    t.data.name: pd.Series(t.data.values, index=ref_times_to_dti(model.ref_date, t.data.index)) for 
    t in project.calibration.targets
}

outputs_to_plot = ["prop_ever_infected"]



#  Plotting seroprevalence data

In [ ]:
def get_serosurvey_data(country: str):
    input_db = get_input_db()
    sero_prevalence = input_db.query(
        table_name='sero-survey',
        conditions={"country": country},
        columns=["serum_pos_prevalence", "estimate_grade", "sampling_start_date", "sampling_end_date"]
    )

    return sero_prevalence

In [ ]:
sero_data = get_serosurvey_data("Malaysia")
print(type(sero_data["sampling_start_date"]))
#filtering the serosurvey studies that are included only during the modelling period
sero_data_filtered = sero_data[sero_data["sampling_start_date"]> model_start_time]



In [ ]:
fig = pyplot.figure(figsize=(15, 12))
for i_out, output in enumerate(outputs_to_plot):
    axis = fig.add_subplot(1, 1, 1)
    if output in targets_dict:
        targets_dict[output].plot(ax=axis, style='.')
    if output in derived_df:
        derived_df[output].plot(ax=axis)
    axis.set_title(output.replace("_", " "))
    #model_start_time = min(sero_data.sampling_start_date)
    #model_end_time = max(sero_data.sampling_start_date)
    axis.set_xlim([model_start_time, model_end_time])
    
    sero_data.plot(ax=axis, kind='scatter',x = 'sampling_start_date', y='serum_pos_prevalence',color='black')
    plt.show()